In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
#%reload_ext autoreload
#%autoreload 2
# %matplotlib inline

In [ ]:
from fastai.vision import *

In [ ]:
path = Path('../input/Kannada-MNIST')
train = pd.read_csv('../input/Kannada-MNIST/train.csv')
test = pd.read_csv('../input/Kannada-MNIST/test.csv')
train_other = pd.read_csv('../input/Kannada-MNIST/Dig-MNIST.csv')

In [ ]:
data,labels = (train.iloc[:,1:],train.iloc[:,0])

In [ ]:
data_other,labels_other = (train_other.iloc[:,1:],train_other.iloc[:,0])

In [ ]:
data_train,labels_train = (pd.concat([data, data_other]),pd.concat([labels, labels_other]))

**Create Train Valid Data Split****

In [ ]:
from sklearn.model_selection import train_test_split

data_train, data_valid, labels_train, labels_valid = train_test_split(data_train, labels_train, test_size=0.25, random_state=42,stratify=labels_train)

In [ ]:
data_train

***Save Images in Folder***

In [ ]:
#import pdb
import imageio
def save_img_to_folder(path:Path,data,labels):
    path.mkdir(parents=True,exist_ok=True)
    
    for i in range(len(data)):
        test = path
        #pdb.set_trace()
        temp_path = test/(str(labels[i]))
        if os.path.isdir(temp_path):
            imageio.imwrite(str(temp_path/(str(i)+'.jpg')), data[i])
            #plt.imsave(str( temp_path/(str(i)+'.jpg') ), data[i], cmap='Greys')
        else:
            temp_path.mkdir(parents=True,exist_ok=True)
            #plt.imsave(str( temp_path/(str(i)+'.jpg') ), data[i], cmap='Greys')
            imageio.imwrite(str(temp_path/(str(i)+'.jpg')), data[i])

In [ ]:
data_arr = np.array(data_valid).reshape(-1,28,28)
labels_arr = np.array(labels_valid)

In [ ]:
save_img_to_folder(Path('valid'),data_arr,labels_arr)

In [ ]:
data_arr1 = np.array(data_train).reshape(-1,28,28)
labels_arr1 = np.array(labels_train)

In [ ]:
save_img_to_folder(Path('train'),data_arr1,labels_arr1)

In [ ]:
path = Path('/kaggle/working')
train_path = path/'train'
valid_path = path/'valid'
path.ls()

In [ ]:
# test_data,test_labels = (test.iloc[:,1:],test.iloc[:,0])

In [ ]:
# data_arr2 = np.array(test_data).reshape(-1,28,28)
# labels_arr2 = np.array(test_labels)

In [ ]:
# save_img_to_folder(Path('test'),data_arr2,labels_arr2)

> **Model Train**

In [ ]:
from fastai.metrics import error_rate
from fastai.vision import *

In [ ]:
np.random.seed(42)
tfms = get_transforms(do_flip=False)

In [ ]:
src = (ImageList.from_folder(path)
        .split_by_folder(train='train', valid='valid')
        .label_from_folder())

In [ ]:
data = (src.transform(tfms,size=64)
        .databunch(bs=64)
        .normalize(mnist_stats))

In [ ]:
# data = (ImageList.from_folder(path)
#         .split_by_folder(train='train', valid='valid')
#         .label_from_folder() 
#         .transform(tfms, size=64)
#         .databunch(bs=64)
#         .normalize(mnist_stats)
#        )

In [ ]:
data.train_ds

In [ ]:
data.classes

In [ ]:
# data = ImageDataBunch.from_folder(path, train="train", valid="valid",test="test",
#         ds_tfms=get_transforms(do_flip=False), size=64,bs=32).normalize(imagenet_stats)

In [ ]:
data.show_batch(rows=4,figsize=(7,8))

In [ ]:
!mkdir -p /tmp/.cache/torch/checkpoints

In [ ]:
!ls /kaggle/input/resnet152

In [ ]:
!cp /kaggle/input/resnet152/resnet152.pth /tmp/.cache/torch/checkpoints/resnet152-b121ed2d.pth

In [ ]:
!ls /tmp/.cache/torch/checkpoints/

In [ ]:
learner = cnn_learner(data,models.resnet152, metrics=[error_rate, accuracy])

In [ ]:
learner.fit_one_cycle(12)

In [ ]:
learner.save('kannada-stage1')

In [ ]:
learner.recorder.plot_losses()

In [ ]:
 interp = ClassificationInterpretation.from_learner(learner)

In [ ]:
losses,idx = interp.top_losses()

In [ ]:
len(data.valid_ds)==len(losses)==len(idx)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
learner.lr_find()

In [ ]:
learner.recorder.plot(suggestion=True)

In [ ]:
learner.unfreeze()
learner.fit_one_cycle(10, max_lr=slice(1e-6,1e-4))

In [ ]:
learner.save('kannada-stage2')

**Tranfer Learning**

In [ ]:
# learner.fit_one_cycle(8,1e-5)

In [ ]:
# learner.save('kannada-stage3')

In [ ]:
# data = (src.transform(tfms, size=64)
#         .databunch().normalize(mnist_stats))

# learner.data = data
# data.train_ds[0][0].shape

In [ ]:
# learner.freeze()

In [ ]:
# learner.lr_find()
# learner.recorder.plot()

In [ ]:
# learner.fit_one_cycle(3, slice(1e-1))

In [ ]:
# learner.save('kannada-stage3')

In [ ]:
# learner.unfreeze()

In [ ]:
# learner.fit_one_cycle(4, max_lr=slice(1e-2, 1e-1/2))

In [ ]:
# learner.recorder.plot_losses()

In [ ]:
# learner.save('kannada-stage4')

In [ ]:
learner.load('kannada-stage2')

In [0]:
img = learner.data.valid_ds[0][0]

In [0]:
learner.predict(img)

In [0]:
learner.data.test_ds

***Predictions on Test Dataset***

In [0]:
#sub_df = pd.DataFrame(columns=['id','label'])

In [0]:
#len(learner.data.test_ds.x)

In [0]:
#learner.predict(learner.data.test_ds.x[6])[1]

In [0]:
# for i in range(len(learner.data.test_ds.x)):
    
#     sub_df.loc[i]=[i+1,int(learner.predict(learner.data.test_ds.x[i])[1])]

In [ ]:
test_csv = pd.read_csv('../input/Kannada-MNIST/test.csv')
test_csv.drop('id',axis = 'columns',inplace = True)
sub_df = pd.DataFrame(columns=['id','label'])
test_data = np.array(test_csv)

In [ ]:
def get_img(data):
    t1 = data.reshape(28,28)/255
    t1 = np.stack([t1]*3,axis=0)
    img = Image(FloatTensor(t1))
    return img

In [ ]:
from fastprogress import progress_bar

In [ ]:
mb=progress_bar(range(test_data.shape[0]))
for i in mb:
    timg=test_data[i]
    img = get_img(timg)
    sub_df.loc[i]=[i+1,int(learner.predict(img)[1])]

In [ ]:
def decr(ido):
    return ido-1

sub_df['id'] = sub_df['id'].map(decr)

In [0]:
sub_df.head()

In [0]:
sub_df.to_csv("submission.csv", index=False)

In [ ]:
!rm -r valid train